<a href="https://colab.research.google.com/github/Alpha25-ai/Smart-cooking-assistant/blob/main/Smart_assistant_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import pandas as pd

db = pd.read_csv("/recipes_ingredients.csv", on_bad_lines="skip", engine="python")


In [ ]:
db.info()

In [ ]:
import pandas as pd

db = pd.read_csv("/recipes.csv", on_bad_lines="skip", engine="python")

def find_recipe(recipe_name):
    result = db[db['Name'].str.lower() == recipe_name.lower()]
    if not result.empty:
        return result.iloc[0]['Ingredients'], result.iloc[0]['Steps']
    else:
        return "Recipe not found!", ""

recipe = input("Enter recipe name: ")
Ingredients, Steps = find_recipe(recipe)

if Ingredients != "Recipe not found!":
    print("\nIngredients:\n", Ingredients)
    print("\nSteps:\n", Steps)
else:
    print("Recipe not found in database!")

Enter recipe name: Sausage Meatloaf OAMC

Ingredients:
 ["ground beef", "ground sausage", "celery chopped", "onion", "saltines", "tomato sauce", "brown sugar", "prepared mustard", "Worcestershire sauce"]

Steps:
 ["Combine ground beef, sausage, celery, onion, saltines, and eggs together and form into 2 loaves.", "Bake covered at 350 degress Fahrenheit in stoneware for 45 to 75  minutes(depending on the shape of your loaves and desired doneness). Drain grease.", "For the sauce, combine tomato sauce, brown sugar, mustard, and Worcestershire sauce.", "Pour sauce over loaves.", "Cover meatloaf and cook 15 minutes.", "Remove cover and cook 10 to 15 minutes.", "To freeze, just freeze the meat mixture in meal size portions. Thaw it in the refrigerator for a day before cooking. You could make and freeze the sauce too, but I usually mix it up when I cook the meat."]


In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
db["text"] = db["Name"].fillna("") + " " + db["Ingredients"].fillna("") + " " + db["Steps"].fillna("")


In [ ]:
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(db["text"])

In [ ]:
def search_recipes(query, top_n=3):
    query_vec = vectorizer.transform([query])   # convert user query into vector
    similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()  # compare with all recipes
    indices = similarity.argsort()[::-1][:top_n]  # get top matches

    results = []
    for i in indices:
        results.append({
            "Name": db.iloc[i]["Name"],
            "Description": db.iloc[i]["Description"],
            "Ingredients": db.iloc[i]["Ingredients"],
            "Ingredients_raw": db.iloc[i]["Ingredients_raw"],
            "Steps": db.iloc[i]["Steps"],
            "Servings": db.iloc[i]["Servings"],
            "Serving_size": db.iloc[i]["Serving_size"],
            "Tags": db.iloc[i]["Tags"],
            "score": similarity[i]
        })
    return results


In [ ]:
print("👩‍🍳 Smart Cooking Assistant Ready! (type 'exit' to quit)")

while True:
    query = input("\nWhat do you want to cook? ")
    if query.lower() == "exit":
        break

    results = search_recipes(query)
    if results:
        print("\nTop Recipes:")
        for r in results:
            print(f"\n🍲 {r['Name']} (score: {r['score']:.2f})")
            print("Description:",r["Description"][:200],"...")
            print("Ingredients:", r["Ingredients"])
            print("Ingredients_raw:", r["Ingredients_raw"])
            print("Steps:", r["Steps"][:200], "...")
            print("Servings",r["Servings"])
            print("Serving_size",r["Serving_size"])
            print("Tags", r["Tags"][:200], "...")

    else:
        print("❌ Sorry, no recipe found.")


👩‍🍳 Smart Cooking Assistant Ready! (type 'exit' to quit)

What do you want to cook? burger

Top Recipes:

🍲 Hot, Sizzling Heat: Lost in the Sahara Burger (score: 0.63)
Description: Inspired by the blazing heat of the desert, the Lost in the Sahara Burger is topped with creamy slices of avocado, a thick wedge of spicy jalapeno Havarti cheese and lots of zesty Bick's Sandwich Save ...
Ingredients: ["toasted pretzel bun", "cooked beef burger", "ketchup", "bick sweet green relish", "red onion", "iceberg lettuce", "jalapeno havarti cheese", "ripened avocados", "spicy pickles", "fresh tomatoes"]
Ingredients_raw: ["1      toasted pretzel bun","1      cooked beef burger","1   teaspoon    ketchup","1   teaspoon   bick\u0027s sweet green relish","1       red onion, slice ","1   leaf    iceberg lettuce","1   slice    jalapeno havarti cheese","4 -5   slices   of ripened avocados","3 -5      bick\u0027s sandwich savers hot and spicy pickles","1      fresh tomatoes, slice "]
Steps: ["Top the bottom 

In [ ]:
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [ ]:
import pandas as pd

db = pd.read_csv("/recipes_ingredients.csv", on_bad_lines="skip", engine="python")


In [ ]:
!pip install fuzzywuzzy

In [ ]:
import pandas as pd
from fuzzywuzzy import process


db = pd.read_csv("/recipes_ingredients.csv", on_bad_lines="skip", engine="python")


def find_recipe(query):

    choices = db['Name'].dropna().tolist()
    best_match, score = process.extractOne(query, choices)

    if score > 60:
        result = db[db['Name'] == best_match].iloc[0]
        return best_match, result["Ingredients"], result["Steps"],result["Description"],result["Ingredients_raw"],result["Servings"],result["Serving_size"],result["Tags"], score
    else:
        return None, None, None, None

print("👩‍🍳 RecipeBot: Hi! Ask me about any recipe (type 'exit' to quit)")

while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("👩‍🍳 RecipeBot: Goodbye! Happy cooking! 🍲")
        break

    name, Ingredients, Steps, Description, Ingredients_raw, Servings, Serving_size, Tags, score = find_recipe(query)

    if Ingredients:
        print(f"\n👩‍🍳 RecipeBot: I think you meant *{name}* (match score: {score})")
        print("\n📝 Ingredients:\n", Ingredients)
        print("\n👨‍🍳 Steps:\n", Steps[:400], "...")
        print("Description:",Description[:200],"...")
        print("Ingredients_raw:", Ingredients_raw)
        print("Servings",Servings)
        print("Serving_size",Serving_size)
        print("Tags", Tags[:200], "...")
    else:
        print("👩‍🍳 RecipeBot: Sorry, I couldn't find a close match. Try another one!")


👩‍🍳 RecipeBot: Hi! Ask me about any recipe (type 'exit' to quit)

You: taxcos

👩‍🍳 RecipeBot: I think you meant *Tacos* (match score: 91)

📝 Ingredients:
 ["corn tortillas", "oil", "minced onion", "minced garlic", "shredded lettuce", "chili powder", "oregano", "cumin", "green peppers minced", "tomato", "shredded cheddar cheese"]

👨‍🍳 Steps:
 ["Fry tortillas in 1/2" hot oil, turning with tongs. Shape each into a U while hot, and set aside in a warm place. Place rest of ingredients except tomato and lettuce in saucepan. Stir and cook 5 minutes or until mixture begins to brown. Add tomatoes and cook 10 minutes longer until mixture thickens. Spoon into fried tortilla shells. Top each with finely shredded lettuce. Also good with grated mil ...
Description: This recipe comes from Melting Pot Potluck. It was put together to announce the arrival of a new pastor and his wife. Origin: Mexico. ...
Ingredients_raw: ["12       corn tortillas","  cooking oil","1   lb    ground chuck","1   tablespoon

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.6/324.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import gradio as gr

db = pd.read_csv("/recipes_ingredients.csv", on_bad_lines="skip", engine="python")

def find_by_ingredient(query):
    results = []
    for i, row in db.iterrows():
        if pd.isna(row["Ingredients"]):
            continue
        if query.lower() in row["Ingredients"].lower():
            results.append({
                "Name": row["Name"],
                "Ingredients": row["Ingredients"],
                "Steps": row["Steps"]
            })
    return results

print("👩‍🍳 RecipeBot: Hi! Ask me 'What can I make with <ingredient>?' (type 'exit' to quit)")

while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("👩‍🍳 RecipeBot: Goodbye! Happy cooking! 🍲")
        break

    words = query.lower().split()
    if "with" in words:
        ingredient = words[words.index("with") + 1]
    else:
        ingredient = query.strip()

    results = find_by_ingredient(ingredient)

    if results:
        print(f"\n👩‍🍳 RecipeBot: Here are some recipes with *{ingredient}*:\n")
        for r in results[:3]:  # show top 3 matches
            print(f"🍲 {r['Name']}")
            print(f"📝 Ingredients: {r['Ingredients']}")
            print(f"👩‍🍳 Steps: {r['Steps'][:250]}...")  # first 250 chars
            print("-" * 50)
    else:
        print("👩‍🍳 RecipeBot: Sorry, I couldn't find recipes with that ingredient.")


👩‍🍳 RecipeBot: Hi! Ask me 'What can I make with <ingredient>?' (type 'exit' to quit)

You: exit
👩‍🍳 RecipeBot: Goodbye! Happy cooking! 🍲


In [ ]:
import gradio as gr
import pandas as pd

db = pd.read_csv("/recipes_ingredients.csv")

def chatbot(message):
    try:

        msg = str(message).lower()

        results = db[db["Ingredients"].astype(str).str.lower().str.contains(msg, na=False)]

        if not results.empty:
            first_recipe = results.iloc[0]
            reply = f"🍲 You can try making **{first_recipe['Name']}**!\n\n"
            reply += f"📝 **Ingredients:**\n{first_recipe['Ingredients']}\n\n"
            reply += f"👩‍🍳 **Steps:**\n{first_recipe['Steps']}"
            return reply
        else:
            return f"⚠️ Sorry, I couldn't find any recipes with '{message}'. Try another ingredient!"

    except Exception as e:
        return f"⚠️ Internal Error: {str(e)}"

demo = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="🍳 Recipe Chatbot",
    description="Ask me what you can cook with any ingredient!"
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://88d4e9986c06ad3538.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
print (db.columns)

Index(['ID', 'Name', 'Description', 'Ingredients', 'Ingredients_raw', 'Steps',
       'Servings', 'Serving_size', 'Tags'],
      dtype='object')


In [ ]:
import gradio as gr
import pandas as pd
import re

db = pd.read_csv("/recipes_ingredients.csv")

def chatbot(message):
    try:
        msg = str(message).lower()

        words = re.findall(r"\b\w+\b", msg)
        ingredients = [w for w in words if len(w) > 3]

        if not ingredients:
            return "🤔 Please mention at least one ingredient."

        mask = db["Ingredients"].astype(str).str.lower()
        matched = db[mask.apply(lambda x: any(ing in x for ing in ingredients))]

        if not matched.empty:
            first_recipe = matched.iloc[0]
            reply = f"🍲 You can make **{first_recipe['Name']}**!\n\n"
            reply += f"📝 **Ingredients:**\n{first_recipe['Ingredients']}\n\n"
            reply += f"👩‍🍳 **Steps:**\n{first_recipe['Steps']}"
            return reply
        else:
            return f"⚠️ Sorry, I couldn't find any recipe with {', '.join(ingredients)}."

    except Exception as e:
        return f"⚠️ Internal Error: {str(e)}"

demo = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="🍳 Recipe Chatbot",
    description="Ask me naturally like: 'What can I make with paneer?'"
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://06642221b1a5847841.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
